## Deliverable 2. Create a Customer Travel Destinations Map.

In [49]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [50]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Zaozerne,UA,2021-04-26 03:45:09,45.1582,33.2787,44.74,60,7,9.13,clear sky
1,1,Ribeira Grande,PT,2021-04-26 03:40:41,38.5167,-28.7000,59.00,72,20,11.50,few clouds
2,2,Yulara,AU,2021-04-26 03:45:09,-25.2406,130.9889,75.20,23,0,14.97,clear sky
3,3,Hermanus,ZA,2021-04-26 03:44:55,-34.4187,19.2345,59.00,92,1,5.01,clear sky
4,4,Codrington,AU,2021-04-26 03:45:10,-38.2667,141.9667,60.84,66,100,11.07,overcast clouds


In [51]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 85


In [52]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.dropna()
preferred_cities_df.head(10)

,City ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
9,9,Butaritari,KI,2021-04-26 03:44:47,3.0707,172.7902,83.17,73,16,11.52,few clouds
12,12,Kaka,TM,2021-04-26 03:45:14,37.3482,59.6143,83.55,18,10,10.40,clear sky
13,13,Ormara,PK,2021-04-26 03:45:14,25.2088,64.6357,83.23,62,2,10.38,clear sky
23,23,Avarua,CK,2021-04-26 03:45:17,-21.2078,-159.7750,80.60,78,20,5.75,few clouds
38,38,Hithadhoo,MV,2021-04-26 03:40:44,-0.6000,73.0833,82.60,67,63,4.47,broken clouds
46,46,Kavieng,PG,2021-04-26 03:45:24,-2.5744,150.7967,84.27,66,95,3.20,overcast clouds
56,56,Santa Marta,CO,2021-04-26 03:45:28,11.2408,-74.1990,84.20,70,75,5.75,broken clouds
57,57,Hambantota,LK,2021-04-26 03:45:28,6.1241,81.1185,84.20,94,20,3.44,few clouds
81,81,Vaini,TO,2021-04-26 03:45:37,-21.2000,-175.2000,80.60,83,40,5.75,light rain
82,82,Nizwa,OM,2021-04-26 03:45:37,22.9333,57.5333,84.40,19,36,3.13,scattered clouds


In [53]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()


City ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [54]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_preferred_cities_df = preferred_cities_df.dropna()


In [55]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Butaritari,KI,83.17,few clouds,3.0707,172.7902,
12,Kaka,TM,83.55,clear sky,37.3482,59.6143,
13,Ormara,PK,83.23,clear sky,25.2088,64.6357,
23,Avarua,CK,80.60,few clouds,-21.2078,-159.7750,
38,Hithadhoo,MV,82.60,broken clouds,-0.6000,73.0833,
46,Kavieng,PG,84.27,overcast clouds,-2.5744,150.7967,
56,Santa Marta,CO,84.20,broken clouds,11.2408,-74.1990,
57,Hambantota,LK,84.20,few clouds,6.1241,81.1185,
81,Vaini,TO,80.60,light rain,-21.2000,-175.2000,
82,Nizwa,OM,84.40,scattered clouds,22.9333,57.5333,


In [56]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Cannot find hotel...skipping.")

Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.


In [57]:
c = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = c.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Butaritari,KI,83.17,few clouds,3.0707,172.7902,Isles Sunset Lodge
13,Ormara,PK,83.23,clear sky,25.2088,64.6357,Crystal Beach and Camping Resort Ormara
23,Avarua,CK,80.60,few clouds,-21.2078,-159.7750,Paradise Inn
38,Hithadhoo,MV,82.60,broken clouds,-0.6000,73.0833,Scoop Guest House
46,Kavieng,PG,84.27,overcast clouds,-2.5744,150.7967,Nusa Island Retreat
56,Santa Marta,CO,84.20,broken clouds,11.2408,-74.1990,Casa de Isabella
57,Hambantota,LK,84.20,few clouds,6.1241,81.1185,Bungalow 63
81,Vaini,TO,80.60,light rain,-21.2000,-175.2000,Keleti Beach Resort
82,Nizwa,OM,84.40,scattered clouds,22.9333,57.5333,Orient Apartments
83,Vaitape,PF,81.39,few clouds,-16.5167,-151.7500,Conrad Bora Bora Nui


In [59]:
# 8a. Create the output File (CSV)

output_data_file = "Weather_Database/WeatherPyy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [61]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [62]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))